# Trend Trader

This notebook is intended to be a practice for handling stock market data and processing it to implement various algorithms(Trend Trading in this case) and improve skills with automation and backtesting. 

**Skills used in ths specific example are:**

1. Data fetching using Yahoo finance
2. Data processing on pandas dataframe, especially rolling windows.
3. Implementing algorithm to execute dummy trades (Just logging for now)
4. Backtesting algorithm using historical data
5. Basic graph using matplotlib for visualisation


## Fetching data from yahoo finance. 

For this example I am going to use data for the stock Asian Paints on NSE. 
I will download all historical EOD data available for Asian Paints

In [2]:
import yfinance as yf

scrip = "ASIANPAINT.NS"
stockData = yf.download(tickers=scrip, period="max", interval="1d")
stockData.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2002-07-01,21.367001,22.466999,21.367001,22.287001,1.578955,87810
2002-07-02,21.370001,22.783001,21.370001,22.520000,1.595461,588615
2002-07-03,22.267000,22.799999,21.933001,22.733000,1.610552,162075
2002-07-04,22.667000,23.333000,22.667000,22.966999,1.627130,266625
2002-07-05,23.400000,25.000000,23.333000,24.056999,1.704352,408600


So there we have all the historical data of Asian Paints right from 2002. Before we go ahead with processing, lets do a basic line plot of the closing prices to visualise if this strategy would at least remotely make sense ot be applied on this stock, Well, I am.. still an amateur... and I dont have tools to automate this process yet, so im just going to eyeball it for now!


In [3]:
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters

# These converters are explicitly used in compliance of a forward looking warning from python.
register_matplotlib_converters()
plt.ylabel('Price')
plt.xlabel('Time')
plt.plot(stockData.index,stockData.Close)


So this looks like a fairly good sample to test the algorithm on, as it has been a fairly up trending stock, but provides good volatility in between to test the whipsaw effect. It will be interesting to see the result. So lets get on with it! 

## Add min and max columns

Calculate rolling 20day max and min values and add respective columns to the dataframe. 

In [7]:
stockData['Max20'] = stockData.Close.rolling(20).max()
stockData['Min20'] = stockData.Close.rolling(20).min()
stockData.tail(10)

,Open,High,Low,Close,Adj Close,Volume,Max20,Min20
Date,,,,,,,,
2020-11-06,2233.949951,2248.000000,2202.500000,2209.050049,2209.050049,2395965,2238.300049,2061.199951
2020-11-09,2249.000000,2249.000000,2214.149902,2225.199951,2225.199951,1691773,2238.300049,2061.199951
2020-11-10,2225.199951,2238.550049,2189.000000,2199.750000,2199.750000,1879561,2238.300049,2061.199951
2020-11-11,2205.000000,2223.000000,2157.300049,2181.699951,2181.699951,2387398,2238.300049,2061.199951
2020-11-12,2172.000000,2201.750000,2162.000000,2171.800049,2171.800049,1626054,2238.300049,2061.199951
2020-11-13,2180.500000,2206.550049,2172.000000,2179.949951,2179.949951,1560060,2238.300049,2070.100098
2020-11-17,2206.399902,2221.899902,2182.000000,2187.649902,2187.649902,1699293,2238.300049,2080.449951
2020-11-18,2185.000000,2210.000000,2166.100098,2195.750000,2195.750000,1817170,2238.300049,2080.449951
2020-11-19,2175.000000,2195.000000,2156.699951,2161.699951,2161.699951,2669522,2238.300049,2080.449951


## Algorithm to trade

The logic followed to record buy and sell signals will be as follows:

1. If price closes above the 20 day max, then buy the scrip
2. If price closes below 20 day min, then sell the scrip


Additional Conditions:

1. Cannot buy if there is already an open position.
2. Cannot sell if there are no open positions.

In [47]:
import pandas as pd
openTrade = False
heading = ['Date','Scrip','Action','Price', 'P/L']
entries = []
for index,row in stockData.iterrows():
    if(openTrade == True):
        if(row.Close <= row.Min20):
            openTrade = False
            sellPrice = row.Close
            tradeEarning = sellPrice - buyPrice
            entry = [index, scrip, 'SELL', sellPrice, tradeEarning]
            entries.append(entry)
            sellPrice = 0
            buyPrice = 0
    elif(openTrade == False):
        if((row.Close >= row.Max20) and row.notnull().any()): 
            openTrade = True
            buyPrice  = row.Close
            entry = [index, scrip, 'BUY', buyPrice]
            entries.append(entry)
    if(index == stockData.index[-1]):
        if(openTrade == True):
            openTrade = False
            sellPrice = row.Close
            tradeEarning = sellPrice - buyPrice
            entry = [index, scrip, 'SELL', sellPrice, tradeEarning]
            entries.append(entry)
            sellPrice = 0
            buyPrice = 0

tradeBook = pd.DataFrame(entries,columns = heading)

tradeBook
        


,Date,Scrip,Action,Price,P/L
0,2002-09-03,ASIANPAINT.NS,BUY,22.913000,NaN
1,2002-10-24,ASIANPAINT.NS,SELL,22.643000,-0.270000
2,2002-12-13,ASIANPAINT.NS,BUY,22.257000,NaN
3,2002-12-23,ASIANPAINT.NS,SELL,21.680000,-0.577000
4,2003-01-14,ASIANPAINT.NS,BUY,22.393000,NaN
...,...,...,...,...,...
153,2020-03-02,ASIANPAINT.NS,SELL,1786.800049,-32.349976
154,2020-04-16,ASIANPAINT.NS,BUY,1743.099976,NaN
155,2020-05-07,ASIANPAINT.NS,SELL,1594.300049,-148.799927
156,2020-06-01,ASIANPAINT.NS,BUY,1693.250000,NaN


So let's do some basic calculations on this output and see how the strategy performed

In [48]:
netPL = tradeBook['P/L'].sum()
#print("Net P/L of the System :", netPL)

totalTrades = tradeBook['P/L'].notnull().sum()
totalWinners = tradeBook['P/L'].gt(0).sum()
totalLosers = tradeBook['P/L'].lt(0).sum()
maxProfit = tradeBook['P/L'].max()
maxLoss = tradeBook['P/L'].min()

lossTrades = [x for x in tradeBook['P/L'] if x < 0]
profitTrades = [x for x in tradeBook['P/L'] if x > 0]
totalProfit = sum(profitTrades)
totalLoss = sum(lossTrades)

avgLoss = totalLoss/totalLosers
avgProfit = totalProfit/totalWinners

hitRatio = totalWinners/totalTrades
edgeRatio = avgProfit/avgLoss*-1


columns = ['Parameter', 'Value']
rows = [["Total Trades",totalTrades],["Total Winners",totalWinners], ["Total Losers",totalLosers],
       ["Max Profit",maxProfit],["Max Loss",maxLoss],["Total Profit",totalProfit],["Total Loss",totalLoss],
       ["Avg Profit",avgProfit],["Avg Loss",avgLoss],["Hit Ratio",hitRatio],
       ["Edge Ratio",edgeRatio],["Net P/L",netPL]]

reportCard = pd.DataFrame(rows,columns = columns)

reportCard




,Parameter,Value
0,Total Trades,79.000000
1,Total Winners,39.000000
2,Total Losers,40.000000
3,Max Profit,515.800049
4,Max Loss,-148.799927
5,Total Profit,1786.509905
6,Total Loss,-996.506056
7,Avg Profit,45.807946
8,Avg Loss,-24.912651
9,Hit Ratio,0.493671


# Conclusion

Looking at the report card above, it seems that this is not a terrible strategy but it is just about holding it's own. It is by no means an implementable strtegy on it's own, and surely needs to be optimised further to make it usable in the larger scheme of things. it needs to be optimised for especially two parameters

1. *Hit ratio* - Although a hit ratio of 0.49 is workable, I would like to aim for a hit ratio better thn 0.55
2. *Edge Ratio* - I would not implement this strategy unless I can achieve an edge ratio of more than 3.0

In order to achieve these intended values, this strategy needs to be optimised to avoid whipsaws, as it is the only way losses are introduced in this particular example. 

## Moving forward

I will try to implement some optimisations to improve the output of this system in the next notebook. I fully intend to use TA_Lib and see what else can be done. 

In case you have any other analysis that you'd like me to do, or suggestions on implementing any optimization please let me know in the comments, and we'll see where we can go with it. 

Also, I would love to hear from you if you have any suggestions to improve this code in ways I can't even imagine at the moment, or point out a mistake that could come and haunt me in the future.

Cheers!